In [2]:
import pandas as pd
import numpy as np

C:\Users\raine\AppData\Local\Temp\ipykernel_23868\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# EXTRACTED MENU ITEMS 

In [3]:
import os
import base64
import re
from anthropic import Anthropic
from collections import defaultdict
import logging # Import logging
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def encode_image(image_path):
    """
    Encode an image file to base64.

    :param image_path: Path to the image file
    :return: Base64 encoded string of the image
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        logging.error(f"Image file not found: {image_path}")
        return None
    except Exception as e:
        logging.error(f"Error encoding image {image_path}: {e}")
        return None

def extract_text_from_image(api_key, image_paths, prompt_text):
    """
    Use Claude to extract text from multiple images.

    :param api_key: Your Anthropic API key
    :param image_paths: List of paths to image files
    :param prompt_text: Text prompt for Claude
    :return: Extracted text from the images
    """
    # Initialize the Anthropic client
    client = Anthropic(api_key=api_key)

    try:
        # Create content array
        content = []

        # Add each image to the content array
        image_added = False
        for image_path in image_paths:
            # Determine media type based on file extension
            media_type = "image/jpeg"  # Default
            if image_path.lower().endswith(".png"):
                media_type = "image/png"
            elif image_path.lower().endswith(".gif"):
                media_type = "image/gif"
            elif image_path.lower().endswith(".webp"):
                 media_type = "image/webp" # Add webp support


            # Encode the image
            logging.info(f"Encoding image: {os.path.basename(image_path)}")
            base64_image = encode_image(image_path)

            if base64_image:
                # Add image to content array
                content.append({
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": media_type,
                        "data": base64_image
                    }
                })
                image_added = True
            else:
                logging.warning(f"Skipping image due to encoding error: {os.path.basename(image_path)}")

        if not image_added:
            logging.error("No images could be successfully encoded and added.")
            return None

        # Add text prompt at the end
        content.append({
            "type": "text",
            "text": prompt_text
        })

        # Send request to Claude
        logging.info("Sending request to Claude API...")
        response = client.messages.create(
            model="claude-3-haiku-20240307", # Using Haiku for speed/cost balance
            # model="claude-3-opus-20240229", # Opus might give better results but is slower/more expensive
            max_tokens=4000, # Increased max_tokens slightly
            messages=[
                {
                    "role": "user",
                    "content": content
                }
            ]
        )

        # Return the extracted text
        extracted_text = response.content[0].text
        logging.info(f"Received response from Claude. Content length: {len(extracted_text)} characters.")
        # Log first few lines of response for debugging
        # logging.debug("Claude Response Head:\n" + "\n".join(extracted_text.split('\n')[:5]))
        return extracted_text

    except Exception as e:
        logging.error(f"An error occurred during Claude API call: {e}", exc_info=True) # Log traceback
        return None

def parse_menu_items(extracted_text):
    """
    Parse menu items, prices, and categories from extracted text.

    :param extracted_text: Text extracted from the image by Claude
    :return: Dictionary of menu items {name: {'price': float, 'category': str}}, plus confidence scores {name: float}
    """
    menu_items = {}
    confidence_scores = {}

    # Basic check if extraction returned anything
    if not extracted_text or not extracted_text.strip():
        logging.warning("Received empty or null text from extraction.")
        return {}, {}

    # Split the text into lines
    lines = extracted_text.strip().split('\n')
    total_non_empty_lines = sum(1 for line in lines if line.strip())
    successful_extractions = 0
    unclear_items = 0
    default_category_items = 0

    logging.info(f"Parsing {total_non_empty_lines} non-empty lines from Claude output.")

    # --- Refined Regex Patterns ---
    # Price pattern: Handles various currency symbols (optional), commas, and decimals
    # Makes currency symbol and spacing optional. Allows integer prices.
    price_pattern = r'(?:[$€£¥]\s*|\b)(\d+(?:,\d{3})*(?:\.\d{1,2})?)\b'
    # Regex to find item name, price, and category (assumes format: NAME PRICE CATEGORY)
    # It tries to be flexible with spacing and potential currency symbols.
    # Group 1: Item Name (non-greedy)
    # Group 2: Price (using price_pattern logic)
    # Group 3: Category (rest of the line)
    # This regex is complex and might need tuning based on Claude's actual output format.
    # Let's try a simpler approach first: find price, then split.

    for line_num, line in enumerate(lines):
        line = line.strip()
        # Skip empty lines or potential headers/footers from Claude
        if not line or line.startswith("Here are the items") or line.startswith("---"):
            continue

        line_confidence = 1.0  # Start with full confidence

        # Check for uncertainty indicators in the text (global line check)
        uncertainty_phrases = ["unclear", "can't make out", "illegible", "not visible", "hard to read", "possibly", "maybe", "appears to be"]
        is_unclear = False
        for phrase in uncertainty_phrases:
            if phrase in line.lower():
                line_confidence *= 0.6  # Reduce confidence if uncertainty is indicated globally
                is_unclear = True
                # break # Found one, no need to check others for this line

        # 1. Find the price first (more reliable anchor)
        price_match = re.search(price_pattern, line)

        if price_match:
            # Extract the price and convert to float
            price_str = price_match.group(1).replace(',', '')
            try:
                price = float(price_str)
            except ValueError:
                logging.warning(f"Line {line_num+1}: Could not convert price '{price_str}' to float. Skipping line: '{line}'")
                continue

            # 2. Extract Item Name (everything before the price match)
            item_name = line[:price_match.start()].strip()
            # Clean up common trailing characters before price
            item_name = re.sub(r'[.…\-_*\s]+$', '', item_name).strip()

            # 3. Extract Category (everything after the price match)
            category_name = line[price_match.end():].strip()
            # Clean up common leading characters after price
            category_name = re.sub(r'^[.…\-_*\s]+', '', category_name).strip()

            # --- Data Cleaning and Validation ---
            # Handle empty item name (likely parsing error or header)
            if not item_name:
                 logging.warning(f"Line {line_num+1}: Extracted empty item name. Skipping line: '{line}'")
                 continue

            # Handle empty or default category
            if not category_name or category_name.lower() in ["unclear", "unknown", "n/a", "none", "-", "--"]:
                category_name = "Uncategorized" # Standardize default
                line_confidence *= 0.8 # Slightly reduce confidence if category was unclear/missing
                default_category_items += 1

            # Remove any "(unclear)" tags added by Claude from name/category
            item_name = item_name.replace("(unclear)", "").strip()
            category_name = category_name.replace("(unclear)", "").strip()

            # If the global 'unclear' flag was set, mark item count
            if is_unclear:
                unclear_items += 1


            # Check name/category quality (very short strings might be noise)
            if len(item_name) < 3:
                line_confidence *= 0.8
            if len(category_name) < 3 and category_name != "Uncategorized": # Allow short defaults
                 line_confidence *= 0.9 # Less penalty for short category

            # Check price reasonableness
            if price <= 0 or price > 1000: # Adjusted range slightly
                line_confidence *= 0.7

            # Store the item
            successful_extractions += 1
            item_key = item_name # Use name as the key

            # Handle duplicate item names (append index if price/category differ)
            if item_key in menu_items:
                 existing_item = menu_items[item_key]
                 # Only create variant if price OR category is different
                 if existing_item['price'] != price or existing_item['category'] != category_name:
                     count = 1
                     new_key = f"{item_name} ({count})"
                     while new_key in menu_items:
                         count += 1
                         new_key = f"{item_name} ({count})"
                     item_key = new_key
                     logging.info(f"Duplicate item name '{item_name}' found with different price/category. Storing as '{item_key}'.")
                 else:
                     # Exact duplicate, maybe increase confidence slightly? Or just skip. Let's skip.
                     logging.info(f"Exact duplicate item found: '{item_name}'. Skipping.")
                     successful_extractions -= 1 # Decrement success counter as we skipped it
                     continue # Skip to next line

            menu_items[item_key] = {'price': price, 'category': category_name}
            confidence_scores[item_key] = max(0.0, min(1.0, line_confidence)) # Clamp confidence 0-1

        else:
            # Line did not contain a recognizable price pattern
            logging.warning(f"Line {line_num+1}: No valid price pattern found. Skipping line: '{line}'")

    # Calculate overall extraction quality metrics
    extraction_rate = successful_extractions / total_non_empty_lines if total_non_empty_lines > 0 else 0
    avg_confidence = sum(confidence_scores.values()) / len(confidence_scores) if confidence_scores else 0

    # Log the extraction metrics
    logging.info(f"Parsing complete. Successfully extracted {successful_extractions} items.")
    logging.info(f"Extraction rate: {extraction_rate:.1%} of non-empty lines.")
    logging.info(f"Items marked unclear by Claude or parser: {unclear_items}")
    logging.info(f"Items assigned default 'Uncategorized' category: {default_category_items}")
    logging.info(f"Average confidence score for extracted items: {avg_confidence:.1%}")

    return menu_items, confidence_scores

def categorize_menu_items(menu_items, num_categories=3):
    """
    Categorize menu items based on their price into groups like A, B, C.

    :param menu_items: Dictionary {name: {'price': float, 'category': str}}
    :param num_categories: Number of price categories (A, B, C...)
    :return: Dictionary {price_category: [{'name': str, 'price': float, 'category': str}, ...]}
    """
    if not menu_items:
        logging.warning("No menu items provided for price categorization.")
        return {}

    # Prepare items list: [{'name': name, 'price': price, 'category': category}, ...]
    items_list = [{'name': name, **data} for name, data in menu_items.items()]

    # Sort items by price (descending)
    sorted_items_list = sorted(items_list, key=lambda x: x['price'], reverse=True)

    # Determine price ranges
    prices = [item['price'] for item in sorted_items_list]
    if not prices: # Should not happen if menu_items is not empty, but safety check
        return {}

    max_price = max(prices)
    min_price = min(prices)
    price_range = max_price - min_price

    logging.info(f"\nPrice range for categorization: ${min_price:.2f} to ${max_price:.2f} (Spread: ${price_range:.2f})")

    # Create categories based on price ranges
    # Result structure: {'A': [item_dict1, item_dict2], 'B': [...]}
    categories = defaultdict(list)

    # Handle edge cases: zero range or only one category requested
    if num_categories <= 1 or price_range == 0:
        logging.info("Assigning all items to Category A (single category requested or zero price range).")
        categories['A'].extend(sorted_items_list)
        return dict(categories) # Convert back to dict from defaultdict

    # --- Adaptive Categorization Logic (from original code) ---
    # Decide whether to use equal price range division or percentile-based division
    large_range_threshold = 3.0 # Use adaptive if range > 3x min price
    use_adaptive = price_range > min_price * large_range_threshold and min_price > 0

    if use_adaptive:
        logging.info(f"Large price range detected (range/min > {large_range_threshold:.1f}) - using adaptive (percentile-based) categorization.")

        num_items = len(sorted_items_list)
        category_boundaries = [] # List of lower bounds for categories B, C, D...

        # Calculate boundary prices based on item count percentiles
        for i in range(1, num_categories):
            idx = int((i * num_items) / num_categories)
            # Use the price of the item at the boundary index as the threshold
            # Ensure index is valid
            if 0 <= idx < num_items:
                 # The boundary is the price *below which* items fall into the next category
                 # So, category A is >= boundary[0], B is < boundary[0] and >= boundary[1], etc.
                 boundary_price = sorted_items_list[idx]['price']
                 # Add a small epsilon if needed to handle exact matches at boundaries consistently
                 # Let's adjust logic: boundary price IS the lower limit for the higher category
                 category_boundaries.append(boundary_price)
            else:
                 # Should not happen with valid indices, but handle gracefully
                 logging.warning(f"Could not determine boundary price for category {chr(65 + i)}")
                 # Use the previous boundary or min price as fallback
                 category_boundaries.append(category_boundaries[-1] if category_boundaries else min_price)

        # Ensure boundaries are unique and sorted descending
        category_boundaries = sorted(list(set(category_boundaries)), reverse=True)

        # Add the minimum price as the effective floor for the last category
        category_boundaries.append(min_price - 0.01) # Ensure min_price items are included

        boundary_strs = [f">${b:.2f}" for b in category_boundaries[:-1]] # Don't print the floor boundary
        logging.info(f"Adaptive category price boundaries (Min price for Cat A, B, C...): {boundary_strs}")

        # Assign items to categories based on these boundaries
        for item_dict in sorted_items_list:
            item_price = item_dict['price']
            assigned = False
            # Find the first boundary the price is >= to
            for i, boundary in enumerate(category_boundaries[:-1]): # Iterate through A, B, C... boundaries
                 if item_price >= boundary:
                     category_letter = chr(65 + i)
                     categories[category_letter].append(item_dict)
                     assigned = True
                     break
            # If not assigned (should only happen for min_price items falling below last explicit boundary)
            if not assigned:
                 # Assign to the last category
                 category_letter = chr(65 + num_categories - 1)
                 categories[category_letter].append(item_dict)

    else:
        # --- Standard Equal Price Range Division ---
        logging.info("Using standard categorization (equal price range division).")
        # Avoid division by zero if num_categories is 0 or less (handled earlier, but safety)
        if num_categories <= 0: num_categories = 1
        category_range_size = price_range / num_categories

        # Print calculated category ranges
        logging.info(f"Calculated category range size: ${category_range_size:.2f}")
        for i in range(num_categories):
             cat_letter = chr(65 + i)
             cat_max = max_price - (i * category_range_size)
             cat_min = max_price - ((i + 1) * category_range_size)
             # Ensure the last category includes the minimum price exactly
             if i == num_categories - 1: cat_min = min_price
             logging.info(f"  Target range for Category {cat_letter}: ~${cat_min:.2f} - ${cat_max:.2f}")


        for item_dict in sorted_items_list:
            item_price = item_dict['price']
            # Determine category index based on price position within the total range
            # Handle max_price edge case: should be in Category A (index 0)
            if item_price == max_price:
                category_index = 0
            # Avoid division by zero if category_range_size is 0 (handled earlier, but safety)
            elif category_range_size > 0:
                 # Calculate how many 'ranges' down from the max price this item is
                 category_index = int((max_price - item_price) / category_range_size)
                 # Clamp index to valid range [0, num_categories - 1]
                 category_index = max(0, min(num_categories - 1, category_index))
            else: # category_range_size is 0 (all prices same) -> should be handled by initial check
                 category_index = 0

            category_letter = chr(65 + category_index)
            categories[category_letter].append(item_dict)

    # Log the distribution of items across final categories
    logging.info("\n--- Final Item Distribution by Price Category ---")
    for category_letter, items in sorted(categories.items()):
        category_min_price = min(item['price'] for item in items) if items else 0
        category_max_price = max(item['price'] for item in items) if items else 0
        logging.info(f"Category {category_letter}: {len(items)} items. Actual price range: ${category_min_price:.2f} - ${category_max_price:.2f}")

    return dict(categories) # Convert back to regular dict


def process_menu_images(api_key, image_folder, num_price_categories=3):
    """
    Process all menu images in a folder, extract items (name, price, category),
    and categorize them by price tiers (A, B, C...).

    :param api_key: Your Anthropic API key
    :param image_folder: Path to folder containing menu images
    :param num_price_categories: Number of price categories (A, B, C...)
    :return: Price-categorized menu items dict, and overall confidence scores dict
    """
    # List all image files in the folder
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.webp'] # Added webp
    image_paths = []

    if not os.path.isdir(image_folder):
        logging.error(f"Image folder not found or is not a directory: {image_folder}")
        return {}, {}

    for file in os.listdir(image_folder):
        if any(file.lower().endswith(ext) for ext in image_extensions):
            image_paths.append(os.path.join(image_folder, file))

    if not image_paths:
        logging.error(f"No supported image files ({', '.join(image_extensions)}) found in {image_folder}")
        return {}, {}

    logging.info(f"Found {len(image_paths)} images to process in folder: {image_folder}")

    # Store all items extracted across all images before final categorization
    # Using the structure: {name: {'price': float, 'category': str}}
    all_menu_items_combined = {}
    all_confidence_scores_combined = {}
    image_extraction_counts = []

    # --- Updated Prompt for Claude ---
    prompt_text = f"""
    Please extract all menu items from the provided image(s).

For each distinct menu item found, provide the following information on a single line:
1. The complete Item Name in English (if available).
2. The Thai name of the item (if available).
3. If only Thai OR only English name is available, just extract what is present and mark the other blank
4. The exact Price (including currency symbol like $ or ฿ if visible, otherwise just the number).
5. The item's Category (e.g., Appetizer, Main Course, Dessert, Beverage, Side Dish).

Format each item STRICTLY as:
English Item Name Thai Item Name $Price Category

Examples (menu item is available in both english and thai):
Classic Caesar Salad ซีซาร์สลัดคลาสสิค $12.99 Appetizer
Grilled Salmon Fillet ปลาแซลมอนย่าง $24.50 Main Course
New York Cheesecake นิวยอร์กชีสเค้ก $8.00 Dessert
Iced Tea ชาเย็น $3.50 Beverage
French Fries เฟรนช์ฟรายส์ $5.00 Side Dish

Examples (menu item is only available in english):
Pad Thai $12.99 Main Course
Classic Caesar Salad $12.99 Appetizer
Grilled Salmon Fillet $24.50 Main Course
New York Cheesecake $8.00 Dessert
Iced Tea $3.50 Beverage
French Fries $5.00 Side Dish

Examples (menu item is only available in thai):
ผัดไทย $12.99 Main Course
ซีซาร์สลัดคลาสสิค $12.99 Appetizer
ปลาแซลมอนย่าง $24.50 Main Course
นิวยอร์กชีสเค้ก $8.00 Dessert
ชาเย็น $3.50 Beverage
เฟรนช์ฟรายส์ $5.00 Side Dish

IMPORTANT INSTRUCTIONS:
- List EVERY visible menu item, and do not include the menu item details.
- If you cannot clearly determine any part (Price or Category), use the placeholder "(unclear)" for that specific part.
- If the category is totally unknown, use "Uncategorized".
- Ensure the price is extracted accurately.
- Do NOT add any introductory text, explanations, summaries, or formatting like bullet points or markdown.
- Only output the list of items in the specified format, one item per line.
- If an item seems to span multiple lines in the menu image, combine it into a single logical item name if possible.
- Pay attention to sections or headers in the menu image to help determine the category.
    """

    # Process images (can be done one-by-one or batched if API/model supports it well)
    # Let's stick to one-by-one for simplicity and robustness against single image failures.
    for idx, image_path in enumerate(image_paths):
        logging.info(f"\n--- Processing image {idx+1}/{len(image_paths)}: {os.path.basename(image_path)} ---")

        extracted_text = extract_text_from_image(api_key, [image_path], prompt_text)

        if extracted_text:
            logging.info(f"Text extraction successful for {os.path.basename(image_path)}. Parsing content...")

            # Parse menu items (name, price, category) from this image's text
            menu_items_single, confidence_scores_single = parse_menu_items(extracted_text)
            item_count_single = len(menu_items_single)
            logging.info(f"Parsed {item_count_single} items from {os.path.basename(image_path)}.")
            image_extraction_counts.append((os.path.basename(image_path), item_count_single))

            # Merge items from this image into the combined dictionary
            for item_name, item_data in menu_items_single.items():
                confidence = confidence_scores_single.get(item_name, 0.5) # Default confidence if missing

                # Check if item already exists in combined list
                if item_name in all_menu_items_combined:
                    existing_data = all_menu_items_combined[item_name]
                    # If price or category differs, create a variant
                    if existing_data['price'] != item_data['price'] or existing_data['category'] != item_data['category']:
                        variant_count = 1
                        variant_name = f"{item_name} (Img {idx+1})" # Add image source to variant name
                        while variant_name in all_menu_items_combined:
                             variant_count += 1
                             variant_name = f"{item_name} (Img {idx+1} Var {variant_count})"

                        logging.warning(f"Item '{item_name}' from {os.path.basename(image_path)} differs from previous entry. Storing as '{variant_name}'. "
                                        f"Old: P={existing_data['price']}, C='{existing_data['category']}'. New: P={item_data['price']}, C='{item_data['category']}'.")
                        all_menu_items_combined[variant_name] = item_data
                        all_confidence_scores_combined[variant_name] = confidence
                    else:
                         # Exact same item found again, potentially update confidence if higher?
                         # For now, let's keep the first encountered confidence.
                         logging.info(f"Item '{item_name}' is an exact duplicate from another image. Skipping.")
                else:
                    # New item, add it
                    all_menu_items_combined[item_name] = item_data
                    all_confidence_scores_combined[item_name] = confidence

            logging.info(f"Combined total unique items so far: {len(all_menu_items_combined)}")
        else:
            logging.error(f"Failed to extract text from {os.path.basename(image_path)}. Skipping this image.")
            image_extraction_counts.append((os.path.basename(image_path), 0)) # Record failure

    # --- Post-Processing and Categorization ---
    logging.info(f"\n===== Image Processing Summary =====")
    total_items_extracted = sum(count for _, count in image_extraction_counts)
    logging.info(f"Total items parsed across all images (before deduplication/variants): {total_items_extracted}")
    logging.info(f"Final unique items/variants stored: {len(all_menu_items_combined)}")

    # Print extraction performance by image
    logging.info("\nItems Parsed Per Image:")
    for image_name, item_count in image_extraction_counts:
        logging.info(f"  {image_name}: {item_count} items")

    # Calculate overall average confidence score
    if all_confidence_scores_combined:
        avg_confidence = sum(all_confidence_scores_combined.values()) / len(all_confidence_scores_combined)
        logging.info(f"Overall average extraction confidence: {avg_confidence:.1%}")
    else:
         logging.info("No confidence scores available.")


    if not all_menu_items_combined:
        logging.error("No menu items were successfully extracted from any images.")
        return {}, {}

    # --- Dynamic Adjustment of Price Categories ---
    # Adjust num_categories based on item count for better distribution
    min_items_per_category = 3 # Aim for at least 3 items per category if possible
    max_possible_categories = len(all_menu_items_combined) // min_items_per_category
    adjusted_categories = max(1, min(num_price_categories, max_possible_categories))

    if adjusted_categories != num_price_categories:
        logging.warning(f"Adjusting number of price categories from {num_price_categories} to {adjusted_categories} "
                      f"based on item count ({len(all_menu_items_combined)}) and target minimum items per category ({min_items_per_category}).")
        num_price_categories = adjusted_categories
    else:
         logging.info(f"Using requested number of price categories: {num_price_categories}")

    # Categorize the combined menu items by price tiers (A, B, C...)
    logging.info("\n--- Categorizing All Extracted Items by Price ---")
    final_price_categories = categorize_menu_items(all_menu_items_combined, num_price_categories)

    return final_price_categories, all_confidence_scores_combined

def main():
    # --- Configuration ---
    # IMPORTANT: Replace with your actual Anthropic API key
    # Consider using environment variables for API keys in production
    API_KEY = os.environ.get('ANTHROPIC_API_KEY') # Looks for env var first

    if not API_KEY:
        logging.error("CRITICAL: Anthropic API key not found. Set the ANTHROPIC_API_KEY environment variable or replace 'YOUR_API_KEY_HERE' in the script.")
        return

    # Path to your folder containing menu images
    IMAGE_FOLDER = './sample/ENG MENUS/Box Office Bar & Café at Prince Theatre Heritage Stay' # Make sure this folder exists and contains images

    # Target number of *price* categories (e.g., A, B, C, D for 4 categories)
    NUM_PRICE_CATEGORIES = 4

    # Output file name
    OUTPUT_FILENAME = "categorized_menu_items.txt"
    # --- End Configuration ---


    logging.info("===== MENU ITEM EXTRACTION AND PRICE CATEGORIZATION =====")
    logging.info(f"Processing menu images from folder: {os.path.abspath(IMAGE_FOLDER)}")
    logging.info(f"Target number of price categories: {NUM_PRICE_CATEGORIES}")

    # Create image folder if it doesn't exist (useful for first run)
    if not os.path.exists(IMAGE_FOLDER):
        try:
            os.makedirs(IMAGE_FOLDER)
            logging.info(f"Created image folder: {IMAGE_FOLDER}")
            logging.warning("Image folder was created, but it's currently empty. Please add menu images to it.")
            return # Stop execution if folder was just created empty
        except OSError as e:
            logging.error(f"Failed to create image folder {IMAGE_FOLDER}: {e}")
            return


    # Process menu images
    price_categories, confidence_scores = process_menu_images(API_KEY, IMAGE_FOLDER, NUM_PRICE_CATEGORIES)

    if price_categories:
        # --- Output Results ---
        menu_rows = []
        for category_letter, items in sorted(price_categories.items()):
            if items:
                # Sort items within the category by price (desc) for writing
                items_sorted = sorted(items, key=lambda x: x['price'], reverse=True)

                for item_dict in items_sorted:
                    menu_rows.append({
                        'Category': item_dict['category'],
                        'Name': item_dict['name'],
                        'Price': item_dict['price']
                    })

        ocr_df = pd.DataFrame(menu_rows, columns=['Category', 'Name', 'Price'])
        return ocr_df
    else:
        logging.error("Processing finished, but no menu items were found or categorized.")

if __name__ == "__main__":
    ocr_df = main()

2025-05-09 00:18:54,075 - INFO - ===== MENU ITEM EXTRACTION AND PRICE CATEGORIZATION =====
2025-05-09 00:18:54,078 - INFO - Processing menu images from folder: c:\wamp64\www\hungryhub_automation\sample\ENG MENUS\Box Office Bar & Café at Prince Theatre Heritage Stay
2025-05-09 00:18:54,079 - INFO - Target number of price categories: 4
2025-05-09 00:18:54,083 - INFO - Found 8 images to process in folder: ./sample/ENG MENUS/Box Office Bar & Café at Prince Theatre Heritage Stay
2025-05-09 00:18:54,085 - INFO - 
--- Processing image 1/8: A4 Drinks_outline_pages-to-jpg-0001.jpg ---
2025-05-09 00:18:55,258 - INFO - Encoding image: A4 Drinks_outline_pages-to-jpg-0001.jpg
2025-05-09 00:18:55,271 - INFO - Sending request to Claude API...
2025-05-09 00:19:01,064 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-05-09 00:19:01,070 - INFO - Received response from Claude. Content length: 1307 characters.
2025-05-09 00:19:01,072 - INFO - Text extraction successf

In [3]:
import os
import base64
import re
from anthropic import Anthropic
from collections import defaultdict
import logging # Import logging
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def encode_image(image_path):
    """
    Encode an image file to base64.

    :param image_path: Path to the image file
    :return: Base64 encoded string of the image
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        logging.error(f"Image file not found: {image_path}")
        return None
    except Exception as e:
        logging.error(f"Error encoding image {image_path}: {e}")
        return None

def extract_text_from_image(api_key, image_paths, prompt_text):
    """
    Use Claude to extract text from multiple images.

    :param api_key: Your Anthropic API key
    :param image_paths: List of paths to image files
    :param prompt_text: Text prompt for Claude
    :return: Extracted text from the images
    """
    # Initialize the Anthropic client
    client = Anthropic(api_key=api_key)

    try:
        # Create content array
        content = []

        # Add each image to the content array
        image_added = False
        for image_path in image_paths:
            # Determine media type based on file extension
            media_type = "image/jpeg"  # Default
            if image_path.lower().endswith(".png"):
                media_type = "image/png"
            elif image_path.lower().endswith(".gif"):
                media_type = "image/gif"
            elif image_path.lower().endswith(".webp"):
                 media_type = "image/webp" # Add webp support


            # Encode the image
            logging.info(f"Encoding image: {os.path.basename(image_path)}")
            base64_image = encode_image(image_path)

            if base64_image:
                # Add image to content array
                content.append({
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": media_type,
                        "data": base64_image
                    }
                })
                image_added = True
            else:
                logging.warning(f"Skipping image due to encoding error: {os.path.basename(image_path)}")

        if not image_added:
            logging.error("No images could be successfully encoded and added.")
            return None

        # Add text prompt at the end
        content.append({
            "type": "text",
            "text": prompt_text
        })

        # Send request to Claude
        logging.info("Sending request to Claude API...")
        response = client.messages.create(
            model="claude-3-haiku-20240307", # Using Haiku for speed/cost balance
            # model="claude-3-opus-20240229", # Opus might give better results but is slower/more expensive
            max_tokens=4000, # Increased max_tokens slightly
            messages=[
                {
                    "role": "user",
                    "content": content
                }
            ]
        )

        # Return the extracted text
        extracted_text = response.content[0].text
        logging.info(f"Received response from Claude. Content length: {len(extracted_text)} characters.")
        # Log first few lines of response for debugging
        # logging.debug("Claude Response Head:\n" + "\n".join(extracted_text.split('\n')[:5]))
        return extracted_text

    except Exception as e:
        logging.error(f"An error occurred during Claude API call: {e}", exc_info=True) # Log traceback
        return None

def parse_menu_items(extracted_text):
    """
    Parse menu items, prices, and categories from extracted text.

    :param extracted_text: Text extracted from the image by Claude
    :return: Dictionary of menu items {name: {'price': float, 'category': str}}, plus confidence scores {name: float}
    """
    menu_items = {}
    confidence_scores = {}

    # Basic check if extraction returned anything
    if not extracted_text or not extracted_text.strip():
        logging.warning("Received empty or null text from extraction.")
        return {}, {}

    # Split the text into lines
    lines = extracted_text.strip().split('\n')
    total_non_empty_lines = sum(1 for line in lines if line.strip())
    successful_extractions = 0
    unclear_items = 0
    default_category_items = 0

    logging.info(f"Parsing {total_non_empty_lines} non-empty lines from Claude output.")

    # --- Refined Regex Patterns ---
    # Price pattern: Handles various currency symbols (optional), commas, and decimals
    # Makes currency symbol and spacing optional. Allows integer prices.
    price_pattern = r'(?:[$€£¥]\s*|\b)(\d+(?:,\d{3})*(?:\.\d{1,2})?)\b'
    # Regex to find item name, price, and category (assumes format: NAME PRICE CATEGORY)
    # It tries to be flexible with spacing and potential currency symbols.
    # Group 1: Item Name (non-greedy)
    # Group 2: Price (using price_pattern logic)
    # Group 3: Category (rest of the line)
    # This regex is complex and might need tuning based on Claude's actual output format.
    # Let's try a simpler approach first: find price, then split.

    for line_num, line in enumerate(lines):
        line = line.strip()
        # Skip empty lines or potential headers/footers from Claude
        if not line or line.startswith("Here are the items") or line.startswith("---"):
            continue

        line_confidence = 1.0  # Start with full confidence

        # Check for uncertainty indicators in the text (global line check)
        uncertainty_phrases = ["unclear", "can't make out", "illegible", "not visible", "hard to read", "possibly", "maybe", "appears to be"]
        is_unclear = False
        for phrase in uncertainty_phrases:
            if phrase in line.lower():
                line_confidence *= 0.6  # Reduce confidence if uncertainty is indicated globally
                is_unclear = True
                # break # Found one, no need to check others for this line

        # 1. Find the price first (more reliable anchor)
        price_match = re.search(price_pattern, line)

        if price_match:
            # Extract the price and convert to float
            price_str = price_match.group(1).replace(',', '')
            try:
                price = float(price_str)
            except ValueError:
                logging.warning(f"Line {line_num+1}: Could not convert price '{price_str}' to float. Skipping line: '{line}'")
                continue

            # 2. Extract Item Name (everything before the price match)
            item_name = line[:price_match.start()].strip()
            # Clean up common trailing characters before price
            item_name = re.sub(r'[.…\-_*\s]+$', '', item_name).strip()

            # 3. Extract Category (everything after the price match)
            category_name = line[price_match.end():].strip()
            # Clean up common leading characters after price
            category_name = re.sub(r'^[.…\-_*\s]+', '', category_name).strip()

            # --- Data Cleaning and Validation ---
            # Handle empty item name (likely parsing error or header)
            if not item_name:
                 logging.warning(f"Line {line_num+1}: Extracted empty item name. Skipping line: '{line}'")
                 continue

            # Handle empty or default category
            if not category_name or category_name.lower() in ["unclear", "unknown", "n/a", "none", "-", "--"]:
                category_name = "Uncategorized" # Standardize default
                line_confidence *= 0.8 # Slightly reduce confidence if category was unclear/missing
                default_category_items += 1

            # Remove any "(unclear)" tags added by Claude from name/category
            item_name = item_name.replace("(unclear)", "").strip()
            category_name = category_name.replace("(unclear)", "").strip()

            # If the global 'unclear' flag was set, mark item count
            if is_unclear:
                unclear_items += 1


            # Check name/category quality (very short strings might be noise)
            if len(item_name) < 3:
                line_confidence *= 0.8
            if len(category_name) < 3 and category_name != "Uncategorized": # Allow short defaults
                 line_confidence *= 0.9 # Less penalty for short category

            # Check price reasonableness
            if price <= 0 or price > 1000: # Adjusted range slightly
                line_confidence *= 0.7

            # Store the item
            successful_extractions += 1
            item_key = item_name # Use name as the key

            # Handle duplicate item names (append index if price/category differ)
            if item_key in menu_items:
                 existing_item = menu_items[item_key]
                 # Only create variant if price OR category is different
                 if existing_item['price'] != price or existing_item['category'] != category_name:
                     count = 1
                     new_key = f"{item_name} ({count})"
                     while new_key in menu_items:
                         count += 1
                         new_key = f"{item_name} ({count})"
                     item_key = new_key
                     logging.info(f"Duplicate item name '{item_name}' found with different price/category. Storing as '{item_key}'.")
                 else:
                     # Exact duplicate, maybe increase confidence slightly? Or just skip. Let's skip.
                     logging.info(f"Exact duplicate item found: '{item_name}'. Skipping.")
                     successful_extractions -= 1 # Decrement success counter as we skipped it
                     continue # Skip to next line

            menu_items[item_key] = {'price': price, 'category': category_name}
            confidence_scores[item_key] = max(0.0, min(1.0, line_confidence)) # Clamp confidence 0-1

        else:
            # Line did not contain a recognizable price pattern
            logging.warning(f"Line {line_num+1}: No valid price pattern found. Skipping line: '{line}'")

    # Calculate overall extraction quality metrics
    extraction_rate = successful_extractions / total_non_empty_lines if total_non_empty_lines > 0 else 0
    avg_confidence = sum(confidence_scores.values()) / len(confidence_scores) if confidence_scores else 0

    # Log the extraction metrics
    logging.info(f"Parsing complete. Successfully extracted {successful_extractions} items.")
    logging.info(f"Extraction rate: {extraction_rate:.1%} of non-empty lines.")
    logging.info(f"Items marked unclear by Claude or parser: {unclear_items}")
    logging.info(f"Items assigned default 'Uncategorized' category: {default_category_items}")
    logging.info(f"Average confidence score for extracted items: {avg_confidence:.1%}")

    return menu_items, confidence_scores


def categorize_menu_items_with_beverage_priority(menu_items):
    """
    Categorize menu items with enhanced beverage detection and price validation.
    
    :param menu_items: Dictionary {name: {'price': float, 'category': str}}
    :return: Dictionary {category_letter: [{'name': str, 'price': float, 'category': str}, ...]}
    """
    if not menu_items:
        logging.warning("No menu items provided for categorization.")
        return {}
    
    # First, clean up any obviously wrong prices
    cleaned_menu_items = {}
    for name, item_data in menu_items.items():
        price = item_data['price']
        
        # Check for unreasonable prices (over 500 baht for a menu item)
        if price > 500:
            logging.warning(f"Found potentially incorrect price for {name}: ${price:.2f}")
            
            # Extract price from item name if possible
            price_match = re.search(r'฿(\d+)', name)
            if price_match:
                corrected_price = float(price_match.group(1))
                logging.info(f"Corrected price from name for {name}: ${corrected_price:.2f}")
                item_data['price'] = corrected_price
        
        cleaned_menu_items[name] = item_data
    
    # Prepare items list
    items_list = [{'name': name, **data} for name, data in cleaned_menu_items.items()]
    
    # Initialize categories
    alcoholic_beverages = []
    non_alcoholic_beverages = []
    other_items = []
    
    # Process each menu item
    for item in items_list:
        name = item['name'].lower()
        category = item['category'].lower() if item['category'] else ''
        
        # DEFINITIVE RULES FOR NON-ALCOHOLIC BEVERAGES
        if ('pepsi' in name or 
            'coca' in name or 
            'sprite' in name or 
            'fanta' in name or 
            'juice' in name or 
            'soda' in name or 
            'water' in name or 
            'tea' in name or 
            'coffee' in name or 
            'น้ำ' in name or  # Thai for water/drink
            'fizz' in name or 
            'mocktail' in name or 
            'peony' in name or 
            'blue asian' in name or 
            'tropical summer' in name or 
            'yu-ney' in name or 
            'freshy island' in name or 
            'chrysanthemum' in name or 
            'grass jelly' in name):
            
            # EXCLUSIONS: Food items with drink words
            if ('soup' in name or 'dessert' in category.lower() or 
                'ginkgo' in name or 'bua loi' in name or 'rice' in name):
                other_items.append(item)
            else:
                # Ensure correct category
                item['category'] = 'Non-Alcoholic Beverage'
                non_alcoholic_beverages.append(item)
            continue
            
        # SPECIAL CASE: Santa Vittoria
        if 'santa vittoria' in name or 'ซานตา วิตตอเรีย' in name:
            # Santa Vittoria is mineral water (non-alcoholic), whether sparkling or still
            item['category'] = 'Non-Alcoholic Beverage'
            non_alcoholic_beverages.append(item)
            continue
            
        # DEFINITIVE RULES FOR ALCOHOLIC BEVERAGES
        if ('beer' in name or 
            'เบียร์' in name or 
            'singha' in name or 
            'tsingtao' in name or 
            'สิงห์' in name or 
            'ชิงเต่า' in name or 
            'wine' in name or 
            'whisky' in name or 
            'whiskey' in name or 
            'vodka' in name or 
            'gin' in name or 
            'rum' in name or 
            'tequila' in name or 
            'dimple' in name):
                
            # Ensure correct category
            item['category'] = 'Alcoholic Beverage'
            alcoholic_beverages.append(item)
            continue
            
        # Use category information as fallback
        if 'beverage' in category:
            if 'alcoholic' in category:
                alcoholic_beverages.append(item)
            else:
                non_alcoholic_beverages.append(item)
        else:
            # Not a beverage or not clearly identified
            other_items.append(item)
    
    # Create result categories dict
    categories = {}
    
    # Assign alcoholic beverages to category Z
    if alcoholic_beverages:
        categories['Z'] = sorted(alcoholic_beverages, key=lambda x: x['price'], reverse=True)
        
    # Assign non-alcoholic beverages to category Y
    if non_alcoholic_beverages:
        categories['Y'] = sorted(non_alcoholic_beverages, key=lambda x: x['price'], reverse=True)
    
    # Apply dynamic price categorization to non-beverage items
    if other_items:
        # Sort by price (descending)
        sorted_items = sorted(other_items, key=lambda x: x['price'], reverse=True)
        
        # Setup thresholds
        expensive_threshold_ratio = 1.3  # 30% difference
        inexpensive_threshold_ratio = 2.0  # 2x difference
        
        # Price below which we switch threshold
        switch_threshold = sorted_items[0]['price'] / 2 if sorted_items else 0
        
        # Keep track of remaining items to categorize
        remaining_items = sorted_items.copy()
        current_category = 0  # Start with category A (0)
        
        while remaining_items:
            current_category_letter = chr(65 + current_category)  # A, B, C, ...
            category_items = []
            
            # Add the first item to the current category
            first_item = remaining_items.pop(0)
            category_items.append(first_item)
            highest_price = lowest_price = first_item['price']
            
            # Determine which threshold to use
            threshold_ratio = expensive_threshold_ratio if highest_price >= switch_threshold else inexpensive_threshold_ratio
            
            # Add more items within threshold
            while remaining_items:
                next_item = remaining_items[0]
                next_price = next_item['price']
                
                if highest_price / next_price > threshold_ratio:
                    break
                
                category_items.append(remaining_items.pop(0))
                lowest_price = next_price
            
            # Store the category
            categories[current_category_letter] = category_items
            
            # Prepare for next category
            current_category += 1
            
            # Adjust threshold if needed
            if remaining_items and remaining_items[0]['price'] < switch_threshold:
                threshold_ratio = inexpensive_threshold_ratio
    
    # Log the categorization results
    logging.info("\n--- Final Categorization Results ---")
    for category_letter, items in sorted(categories.items()):
        category_type = "Alcoholic Beverages" if category_letter == 'Z' else \
                       "Non-Alcoholic Beverages" if category_letter == 'Y' else \
                       f"Price Category"
        price_range = f"${min(item['price'] for item in items):.2f} - ${max(item['price'] for item in items):.2f}" if items else "N/A"
        logging.info(f"Category {category_letter} ({category_type}): {len(items)} items. Price range: {price_range}")
        for item in items:
            logging.info(f"  ${item['price']:.2f} - {item['name']}")
    
    return categories


def process_menu_images(api_key, image_folder, num_price_categories=3):
    """
    Process all menu images in a folder, extract items (name, price, category),
    and categorize them by price tiers (A, B, C...).

    :param api_key: Your Anthropic API key
    :param image_folder: Path to folder containing menu images
    :param num_price_categories: Number of price categories (A, B, C...)
    :return: Price-categorized menu items dict, and overall confidence scores dict
    """
    # List all image files in the folder
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.webp'] # Added webp
    image_paths = []

    if not os.path.isdir(image_folder):
        logging.error(f"Image folder not found or is not a directory: {image_folder}")
        return {}, {}

    for file in os.listdir(image_folder):
        if any(file.lower().endswith(ext) for ext in image_extensions):
            image_paths.append(os.path.join(image_folder, file))

    if not image_paths:
        logging.error(f"No supported image files ({', '.join(image_extensions)}) found in {image_folder}")
        return {}, {}

    logging.info(f"Found {len(image_paths)} images to process in folder: {image_folder}")

    # Store all items extracted across all images before final categorization
    # Using the structure: {name: {'price': float, 'category': str}}
    all_menu_items_combined = {}
    all_confidence_scores_combined = {}
    image_extraction_counts = []

    # --- Updated Prompt for Claude ---
    prompt_text = f"""
    Please extract all menu items from the provided image(s).

For each distinct menu item found, provide the following information on a single line:
1. The complete Item Name in English (if available).
2. The Thai name of the item (if available).
3. If only Thai OR only English name is available, just extract what is present and mark the other blank
4. The exact Price (including currency symbol like $ or ฿ if visible, otherwise just the number).
5. The item's Category (e.g., Appetizer, Main Course, Dessert, Beverage, Side Dish).

Format each item STRICTLY as:
English Item Name Thai Item Name $Price Category

Examples (menu item is available in both english and thai):
Classic Caesar Salad ซีซาร์สลัดคลาสสิค $12.99 Appetizer
Grilled Salmon Fillet ปลาแซลมอนย่าง $24.50 Main Course
New York Cheesecake นิวยอร์กชีสเค้ก $8.00 Dessert
Iced Tea ชาเย็น $3.50 Beverage
French Fries เฟรนช์ฟรายส์ $5.00 Side Dish

Examples (menu item is only available in english):
Pad Thai $12.99 Main Course
Classic Caesar Salad $12.99 Appetizer
Grilled Salmon Fillet $24.50 Main Course
New York Cheesecake $8.00 Dessert
Iced Tea $3.50 Beverage
French Fries $5.00 Side Dish

Examples (menu item is only available in thai):
ผัดไทย $12.99 Main Course
ซีซาร์สลัดคลาสสิค $12.99 Appetizer
ปลาแซลมอนย่าง $24.50 Main Course
นิวยอร์กชีสเค้ก $8.00 Dessert
ชาเย็น $3.50 Beverage
เฟรนช์ฟรายส์ $5.00 Side Dish

IMPORTANT INSTRUCTIONS:
- List EVERY visible menu item, and do not include the menu item details.
- If you cannot clearly determine any part (Price or Category), use the placeholder "(unclear)" for that specific part.
- If the category is totally unknown, use "Uncategorized".
- Ensure the price is extracted accurately.
- Do NOT add any introductory text, explanations, summaries, or formatting like bullet points or markdown.
- Only output the list of items in the specified format, one item per line.
- If an item seems to span multiple lines in the menu image, combine it into a single logical item name if possible.
- Pay attention to sections or headers in the menu image to help determine the category.

CRITICAL BEVERAGE CATEGORIZATION INSTRUCTIONS:
- You MUST correctly identify ALL beverages and categorize them properly:
  * ALL beers (Singha, Tsingtao, Chang, etc.) MUST be labeled as "Alcoholic Beverage"
  * ALL wines, spirits, whiskeys, cocktails MUST be labeled as "Alcoholic Beverage"
  * ALL sodas, juices, teas, coffees, mocktails MUST be labeled as "Non-Alcoholic Beverage"
  * Santa Vittoria Sparkling is alcoholic, regular Santa Vittoria is non-alcoholic water

EXAMPLES OF CORRECT BEVERAGE CATEGORIZATION:
Tsingtao Beer 青岛啤酒 $130.00 Alcoholic Beverage
Singha Beer สิงห์เบียร์ $110.00 Alcoholic Beverage
Pepsi $30.00 Non-Alcoholic Beverage
Orange Juice $80.00 Non-Alcoholic Beverage
Coffee ชาเย็น $75.00 Non-Alcoholic Beverage
Wine ไวน์ $220.00 Alcoholic Beverage
    """

    # Process images (can be done one-by-one or batched if API/model supports it well)
    # Let's stick to one-by-one for simplicity and robustness against single image failures.
    for idx, image_path in enumerate(image_paths):
        logging.info(f"\n--- Processing image {idx+1}/{len(image_paths)}: {os.path.basename(image_path)} ---")

        extracted_text = extract_text_from_image(api_key, [image_path], prompt_text)

        if extracted_text:
            logging.info(f"Text extraction successful for {os.path.basename(image_path)}. Parsing content...")

            # Parse menu items (name, price, category) from this image's text
            menu_items_single, confidence_scores_single = parse_menu_items(extracted_text)
            item_count_single = len(menu_items_single)
            logging.info(f"Parsed {item_count_single} items from {os.path.basename(image_path)}.")
            image_extraction_counts.append((os.path.basename(image_path), item_count_single))

            # Merge items from this image into the combined dictionary
            for item_name, item_data in menu_items_single.items():
                confidence = confidence_scores_single.get(item_name, 0.5) # Default confidence if missing

                # Check if item already exists in combined list
                if item_name in all_menu_items_combined:
                    existing_data = all_menu_items_combined[item_name]
                    # If price or category differs, create a variant
                    if existing_data['price'] != item_data['price'] or existing_data['category'] != item_data['category']:
                        variant_count = 1
                        variant_name = f"{item_name} (Img {idx+1})" # Add image source to variant name
                        while variant_name in all_menu_items_combined:
                             variant_count += 1
                             variant_name = f"{item_name} (Img {idx+1} Var {variant_count})"

                        logging.warning(f"Item '{item_name}' from {os.path.basename(image_path)} differs from previous entry. Storing as '{variant_name}'. "
                                        f"Old: P={existing_data['price']}, C='{existing_data['category']}'. New: P={item_data['price']}, C='{item_data['category']}'.")
                        all_menu_items_combined[variant_name] = item_data
                        all_confidence_scores_combined[variant_name] = confidence
                    else:
                         # Exact same item found again, potentially update confidence if higher?
                         # For now, let's keep the first encountered confidence.
                         logging.info(f"Item '{item_name}' is an exact duplicate from another image. Skipping.")
                else:
                    # New item, add it
                    all_menu_items_combined[item_name] = item_data
                    all_confidence_scores_combined[item_name] = confidence

            logging.info(f"Combined total unique items so far: {len(all_menu_items_combined)}")
        else:
            logging.error(f"Failed to extract text from {os.path.basename(image_path)}. Skipping this image.")
            image_extraction_counts.append((os.path.basename(image_path), 0)) # Record failure

    # --- Post-Processing and Categorization ---
    logging.info(f"\n===== Image Processing Summary =====")
    total_items_extracted = sum(count for _, count in image_extraction_counts)
    logging.info(f"Total items parsed across all images (before deduplication/variants): {total_items_extracted}")
    logging.info(f"Final unique items/variants stored: {len(all_menu_items_combined)}")

    # Print extraction performance by image
    logging.info("\nItems Parsed Per Image:")
    for image_name, item_count in image_extraction_counts:
        logging.info(f"  {image_name}: {item_count} items")

    # Calculate overall average confidence score
    if all_confidence_scores_combined:
        avg_confidence = sum(all_confidence_scores_combined.values()) / len(all_confidence_scores_combined)
        logging.info(f"Overall average extraction confidence: {avg_confidence:.1%}")
    else:
         logging.info("No confidence scores available.")


    if not all_menu_items_combined:
        logging.error("No menu items were successfully extracted from any images.")
        return {}, {}

    # --- Dynamic Adjustment of Price Categories ---
    # Adjust num_categories based on item count for better distribution
    # min_items_per_category = 3 # Aim for at least 3 items per category if possible
    # max_possible_categories = len(all_menu_items_combined) // min_items_per_category
    # adjusted_categories = max(1, min(num_price_categories, max_possible_categories))

    # if adjusted_categories != num_price_categories:
    #     logging.warning(f"Adjusting number of price categories from {num_price_categories} to {adjusted_categories} "
    #                   f"based on item count ({len(all_menu_items_combined)}) and target minimum items per category ({min_items_per_category}).")
    #     num_price_categories = adjusted_categories
    # else:
    #      logging.info(f"Using requested number of price categories: {num_price_categories}")

    # Categorize the combined menu items by price tiers (A, B, C...)
    logging.info("\n--- Categorizing All Extracted Items by Price ---")
    final_categories = categorize_menu_items_with_beverage_priority(all_menu_items_combined)

    return final_categories, all_confidence_scores_combined

def main():
    # --- Configuration ---
    # IMPORTANT: Replace with your actual Anthropic API key
    # Consider using environment variables for API keys in production
    API_KEY = os.environ.get('ANTHROPIC_API_KEY') # Looks for env var first

    if not API_KEY:
        logging.error("CRITICAL: Anthropic API key not found. Set the ANTHROPIC_API_KEY environment variable or replace 'YOUR_API_KEY_HERE' in the script.")
        return

    # Path to your folder containing menu images
    IMAGE_FOLDER = './sample/ENG MENUS/Box Office Bar & Café at Prince Theatre Heritage Stay' # Make sure this folder exists and contains images

    # Output file name
    OUTPUT_FILENAME = "categorized_menu_items.txt"
    # --- End Configuration ---


    logging.info("===== MENU ITEM EXTRACTION AND PRICE CATEGORIZATION =====")
    logging.info(f"Processing menu images from folder: {os.path.abspath(IMAGE_FOLDER)}")

    # Create image folder if it doesn't exist (useful for first run)
    if not os.path.exists(IMAGE_FOLDER):
        try:
            os.makedirs(IMAGE_FOLDER)
            logging.info(f"Created image folder: {IMAGE_FOLDER}")
            logging.warning("Image folder was created, but it's currently empty. Please add menu images to it.")
            return # Stop execution if folder was just created empty
        except OSError as e:
            logging.error(f"Failed to create image folder {IMAGE_FOLDER}: {e}")
            return


    # Process menu images
    price_categories, confidence_scores = process_menu_images(API_KEY, IMAGE_FOLDER)

    if price_categories:
        # --- Output Results ---
        menu_rows = []
        for category_letter, items in sorted(price_categories.items()):
            if items:
                # Sort items within the category by price (desc) for writing
                items_sorted = sorted(items, key=lambda x: x['price'], reverse=True)

                for item_dict in items_sorted:
                    menu_rows.append({
                        'Category': item_dict['category'],
                        'Name': item_dict['name'],
                        'Price': item_dict['price']
                    })

        ocr_df = pd.DataFrame(menu_rows, columns=['Category', 'Name', 'Price'])
        return ocr_df
    else:
        logging.error("Processing finished, but no menu items were found or categorized.")

if __name__ == "__main__":
    ocr_df = main()


2025-05-09 00:33:50,535 - INFO - ===== MENU ITEM EXTRACTION AND PRICE CATEGORIZATION =====
2025-05-09 00:33:50,537 - INFO - Processing menu images from folder: c:\wamp64\www\hungryhub_automation\sample\ENG MENUS\Box Office Bar & Café at Prince Theatre Heritage Stay
2025-05-09 00:33:50,542 - INFO - Found 8 images to process in folder: ./sample/ENG MENUS/Box Office Bar & Café at Prince Theatre Heritage Stay
2025-05-09 00:33:50,546 - INFO - 
--- Processing image 1/8: A4 Drinks_outline_pages-to-jpg-0001.jpg ---
2025-05-09 00:33:51,723 - INFO - Encoding image: A4 Drinks_outline_pages-to-jpg-0001.jpg
2025-05-09 00:33:51,737 - INFO - Sending request to Claude API...
2025-05-09 00:33:55,715 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-05-09 00:33:55,722 - INFO - Received response from Claude. Content length: 709 characters.
2025-05-09 00:33:55,724 - INFO - Text extraction successful for A4 Drinks_outline_pages-to-jpg-0001.jpg. Parsing content...
2025

In [4]:
ocr_df

,Category,Name,Price
0,++ Includes The Following Cocktails and Chang ...,Cocktails Free Flow,499.0
1,Strong and Boozy,Long Island E. T,300.0
2,Strong and Boozy,La Dolce Vita (Negroni),300.0
3,Milkshake,Lushmore ฿,300.0
4,Classics,The Dude (White Russian),280.0
...,...,...,...
92,Alcoholic Beverage,Chang (Small),120.0
93,Alcoholic Beverage,Chang - Glass (330ml),100.0
94,Alcoholic Beverage,Rapeerumpant ฿,80.0
95,Alcoholic Beverage,Ginger Ale ฿,40.0


In [ ]:
import re

# Remove all characters except letters and numbers, then lowercase
ocr_df['Preprocessed_Name'] = ocr_df['Name'].str.replace(r'[^a-zA-Z0-9]฿', '', regex=True).str.lower().str.strip()

ocr_df


,Category,Name,Price,Preprocessed_Name
0,++ Includes The Following Cocktails and Chang ...,Cocktails Free Flow,499.0,cocktails free flow
1,Strong and Boozy,Long Island E. T,300.0,long island e. t
2,Strong and Boozy,La Dolce Vita (Negroni),300.0,la dolce vita (negroni)
3,Milkshake,Lushmore ฿,300.0,lushmore
4,Classics,The Dude (White Russian),280.0,the dude (white russian)
...,...,...,...,...
92,Alcoholic Beverage,Chang (Small),120.0,chang (small)
93,Alcoholic Beverage,Chang - Glass (330ml),100.0,chang - glass (330ml)
94,Alcoholic Beverage,Rapeerumpant ฿,80.0,rapeerumpant
95,Alcoholic Beverage,Ginger Ale ฿,40.0,ginger ale


# ACTUAL MENU 

In [18]:
actual_df = pd.read_csv('./sample/ENG MENUS/Box Office Bar - Prince Theatre Heritage Stay Hostel Manual.csv')

actual_df.drop(columns=['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'], inplace=True)

actual_df['Preprocessed_Name'] = actual_df['Name'].str.replace(r'[^a-zA-Z0-9]฿', '', regex=True)
actual_df['Preprocessed_Name'] = actual_df['Name'].str.replace(r'.', '', regex=False).str.lower().str.strip()

actual_df

,Category,Name,Description,Price,Note,Preprocessed_Name
0,BOTTLED AND CANNED BEER,Chang (Small).,NaN,120.0,plus vat & service 10%,chang (small)
1,NaN,Chang (Large).,NaN,150.0,NaN,chang (large)
2,NaN,Chang Cold Brew (XL Can).,NaN,150.0,NaN,chang cold brew (xl can)
3,NaN,Singha (Small).,NaN,130.0,NaN,singha (small)
4,NaN,Singha (Large),NaN,160.0,NaN,singha (large)
...,...,...,...,...,...,...
264,NaN,NaN,NaN,NaN,NaN,NaN
265,NaN,NaN,NaN,NaN,NaN,NaN
266,NaN,NaN,NaN,NaN,NaN,NaN
267,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Merge on both Name and Price for exact match
comparison = pd.merge(actual_df, ocr_df, on=['Preprocessed_Name', 'Price'], how='outer', indicator=True)

# Number of exact matches (Name and Price)
num_matches = (comparison['_merge'] == 'both').sum()
total_truth = len(actual_df)

accuracy = num_matches / total_truth
print(f"OCR Name & Price Accuracy: {accuracy:.2%}")

OCR Name & Price Accuracy: 19.33%
